In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from src import caa_survey_utils, config

from src.old_mappers.ModeConditionMapperV4 import ModeConditionMapper as ModeConditionMapperV4
from src.old_mappers.ModeConditionMapperV4_Corrected import ModeConditionMapper as ModeConditionMapperV4_Corrected
from src.old_mappers.ModeConditionMapperV5 import ModeConditionMapper as ModeConditionMapperV5
from src.old_mappers.ModeConditionMapperV6 import ModeConditionMapper as ModeConditionMapperV6
from src.old_mappers.ModeConditionMapperV6_Old_LASAM_Mode_LU import ModeConditionMapper as ModeConditionMapperV6_old_lasam_mode_lu


%load_ext autoreload
%autoreload 2
pd.set_option('display.float_format', '{:.2f}'.format)

# 1. Import and Preprocess Data

## 1.1 CAA Survey Data

In [2]:
# CAA for Heathrow Airport
caa_2023_24_path = r"\\uk-lon-FAS02\Projects\UNIF\Projects\60H700SA - Heathrow SAS 2024\04 Technical\03 LASAM Development\11_Mode choice models\21 Origin assignment\01 Sent to TP\Full dataset for LASAM Zone Assignment.xlsx"
caa_original = pd.read_excel(caa_2023_24_path, engine='openpyxl')
caa_original = caa_original.drop(columns=['Date', 'Mode Group', 'Rename', 'Tylers proposed change', 'Exclusion', 'Report Method Timestamp', 'Weighting'])

### Remove Dummy Records and uplift remaining population

In [3]:
caa_lhr = caa_original[(caa_original['AIRPORT_Prefix']=='LHR')].copy()
caa_lhr.reset_index(drop=True, inplace=True)
caa_lhr = caa_survey_utils.process_dummy_records(caa_lhr)

caa_lhr.head()

0 dummy records removed and reminaing population uplifted by 1.0


,TAG,AIRPORT_Prefix,APT_TERMINAL,GATE,Year,SYSTEM_QUARTER,SYSTEM_MONTH,IDATE,SYSTEM_START_TIME,SEX,...,SYSTEM_PURPOSE1,PURPOSE,MBUSINESS,PROMOTE,ETHNIC,DISABLE,POP,DUMMY_FLAG,RUN_DATE,NATION1
0,Heathrow-11/01/2024-TZR123-330,LHR,4,NaN,2024,1,Jan,2024-01-11,13:49:35,Male,...,Leisure,Leisure Other,NaN,08,White - Irish,No,1561.89,Missing,27/03/2025 15:07:51,Ireland
1,Heathrow-14/01/2024-IER123-328,LHR,3,NaN,2024,1,Jan,2024-01-14,06:24:19,Female,...,Business,Business,NaN,10,Other,No,809.01,Missing,27/03/2025 15:07:51,New Zealand
2,Heathrow-05/01/2024-EFL123-326,LHR,5,NaN,2024,1,Jan,2024-01-05,10:10:41,Female,...,Leisure,Leisure Other,NaN,07,White - Any other White background,No,1058.39,Missing,27/03/2025 15:07:51,United States of America (USA)
3,Heathrow-05/01/2024-EAU123-326,LHR,3,NaN,2024,1,Jan,2024-01-05,20:34:22,Male,...,Leisure,Visiting Friends and Relatives,NaN,06,Asian - Chinese,No,849.12,Missing,27/03/2025 15:07:51,China
4,Heathrow-10/01/2024-IER123-314,LHR,2,NaN,2024,1,Jan,2024-01-10,17:04:42,Male,...,Leisure,Visiting Friends and Relatives,NaN,08,White - Any other White background,No,688.41,Missing,27/03/2025 15:07:51,Sweden


### Remove records for interline passengers

In [4]:
caa_lhr = caa_survey_utils.remove_interline_pax(caa_lhr)

removed 0 rows with interline passengers


### Assign LASAM Segment

In [5]:
caa_lhr = pd.merge(caa_lhr, config.segment_lu, on=['SYSTEM_COUNTRY', 'SYSTEM_RouteTo', 'SYSTEM_PURPOSE1', 'SYSTEM_Market'], how='left')

### Some re-naming

In [6]:
# update mode fields from TfL Rail to Elizabeth Line
columns_to_update = ['MODEA', 'MODEB', 'MODEC', 'SYSTEM_FINALMODE']
caa_lhr[columns_to_update] = caa_lhr[columns_to_update].replace('TfL Rail (formerly Heathrow Connect)', 'Elizabeth Line')

# update column name
caa_lhr = caa_lhr.rename(columns={'APT_TERMINAL': 'Terminal'})

### Assign LASAM mode based on CAA final mode to compare mode shares

In [7]:
caa_lhr = pd.merge(caa_lhr, config.caa_final_mode_lasam_mode_lu, on='SYSTEM_FINALMODE', how='left')

### Convert A B C mode categories that are in CAA to Last, 2ndLast and 3rdLast

In [ ]:
caa_lhr['Last'] = caa_lhr.apply(caa_survey_utils.apply_last_mode, axis=1)

caa_lhr['2ndLast'] = caa_lhr.apply(caa_survey_utils.apply_2ndlast_mode, axis=1)

caa_lhr['3rdLast'] = caa_lhr.apply(caa_survey_utils.apply_3rdlast_mode, axis=1)

### Add Origin column to classify the origin into LDN, NonLDN and Airport

In [9]:
caa_lhr['Origin'] = caa_lhr.apply(lambda row: 'AIRPORT' if row['SYSTEM_District'] in ['Heathrow Airport (SE)']
                                 else ('LDN' if row['SYSTEM_County']=='Greater London' else 'NonLDN'), axis=1)

## Add columns that indicate whether certain modes have been used at least once

In [ ]:
# column to flag that elizabeth line has been used at least once
caa_lhr['Contains_Elizabeth_Line'] = caa_lhr.apply(caa_survey_utils.apply_contains_mode, axis=1, mode='Elizabeth Line')

# column to flag that Heathrow Express has been used at least once
caa_lhr['Contains_Heathrow_Express'] = caa_lhr.apply(caa_survey_utils.apply_contains_mode, axis=1, mode='Heathrow Express')

# column to flag that the Tube has been used at least once
caa_lhr['Contains_Tube'] = caa_lhr.apply(caa_survey_utils.apply_contains_mode, axis=1, mode='Tube/Metro/Subway')

# column to flat that a rental car has been used at least once
caa_lhr['Contains_Rental'] = caa_lhr.apply(caa_survey_utils.apply_contains_mode, axis=1, mode=['Rental car - short term car park', 'Rental car - hire car courtesy bus'])

# 2. Assign LASAM Modes

In [11]:
caa_lhr_2024 = caa_lhr[caa_lhr.Year == 2024].copy()

In [12]:
caa_lhr_mapped_v4 = ModeConditionMapperV4(caa_lhr_2024).main_run_all()
caa_lhr_mapped_v4_corrected = ModeConditionMapperV4_Corrected(caa_lhr_2024).main_run_all()
caa_lhr_mapped_v5 = ModeConditionMapperV5(caa_lhr_2024).main_run_all()
caa_lhr_mapped_v6 = ModeConditionMapperV6(caa_lhr_2024).main_run_all()
caa_lhr_mapped_v6_old_lasam_mode_lu = ModeConditionMapperV6_old_lasam_mode_lu(caa_lhr_2024).main_run_all()

# 3. Summarise all the different mode assignments

These are compared to each other in a separate spreadsheet here "\\GBLON7VS01.europe.jacobs.com\Projects\UNIF\Projects\60H700SA - Heathrow SAS 2024\04 Technical\03 LASAM Development\2024 Base Mtx\Mode Conditions Rework for LASAM v4.7\Mode Assignment Version Comparison.xlsx"

In [13]:
def get_system_final_mode_summary(caa_df: pd.DataFrame) -> pd.DataFrame:
    caa_df = caa_df.copy()
    caa_finalmode_sums = caa_df[['SYSTEM_FINALMODE', 'POP']].groupby('SYSTEM_FINALMODE').sum().reset_index()
    caa_finalmode_sums['prop'] = caa_finalmode_sums.POP / caa_finalmode_sums.POP.sum() * 100
    return caa_finalmode_sums

get_system_final_mode_summary(caa_lhr_2024)

,SYSTEM_FINALMODE,POP,prop
0,Airline courtesy car,57087.81,0.09
1,Airport to airport coach service,30828.24,0.05
2,Bus Unspecified,117378.09,0.19
3,Bus/coach company unknown,52149.20,0.09
4,Car Unspecified,21865.31,0.04
5,Charter coach,672967.09,1.10
6,Chauffer,353478.39,0.58
7,Courtesy bus (travel agent),17730.94,0.03
8,Cycle,2288.66,0.00
9,Elizabeth Line,7554967.91,12.33


Get the number of each LASAM mode based directly on the final mode

In [14]:
def get_lasam_final_mode_summary(caa_df: pd.DataFrame) -> pd.DataFrame:
    caa_df = caa_df.copy()

    caa_finalmode_lasam_mode_sums = caa_df[['SYSTEM_FINALMODE', 'POP']].copy()
    caa_finalmode_lasam_mode_sums = caa_finalmode_lasam_mode_sums.merge(config.caa_mode_allocation_lasam_mode_lu[['Mode_Allocated', 'LASAM_Mode']], left_on='SYSTEM_FINALMODE', right_on='Mode_Allocated')
    caa_finalmode_lasam_mode_sums = caa_finalmode_lasam_mode_sums[['LASAM_Mode', 'POP']].groupby('LASAM_Mode').sum().reset_index()
    caa_finalmode_lasam_mode_sums['prop'] = caa_finalmode_lasam_mode_sums.POP / caa_finalmode_lasam_mode_sums.POP.sum() * 100

    return caa_finalmode_lasam_mode_sums

get_lasam_final_mode_summary(caa_lhr_2024)

,LASAM_Mode,POP,prop
0,BR Coach (RailAir Coach),485985.77,0.81
1,Bus/Coach,4492153.96,7.48
2,Charter Coach,672967.09,1.12
3,Crossrail,7554967.91,12.59
4,Heathrow Express,2808945.53,4.68
5,Kiss-and-Fly,9955969.91,16.59
6,London Underground,8994863.10,14.99
7,Other,66463.07,0.11
8,Park-and-Fly,6093811.15,10.15
9,Taxi,18896873.76,31.48


In [46]:
def get_assigned_mode_summary(caa_df: pd.DataFrame) -> pd.DataFrame:
    caa_df = caa_df.copy()
    lasam_mode_col = 'LASAM_Mode' if 'LASAM_Mode' in caa_df.columns else 'LASAM Mode'
    summary_df = caa_df[['POP', lasam_mode_col]].groupby(lasam_mode_col).sum().reset_index()
    summary_df['prop'] = summary_df.POP / summary_df.POP.sum() * 100
    return summary_df

In [47]:
get_assigned_mode_summary(caa_lhr_mapped_v4)

,LASAM Mode,POP,prop
0,BR Coach (RailAir Coach),153139.76,0.25
1,Bus/Coach,4915002.28,8.02
2,Charter Coach,696959.33,1.14
3,Crossrail,7566175.10,12.35
4,Heathrow Express,2862047.20,4.67
5,Kiss-and-Fly,10193198.56,16.64
6,London Underground,8981204.10,14.66
7,Other,414137.43,0.68
8,Park-and-Fly,6861577.03,11.20
9,Taxi,18607540.10,30.38


In [48]:
get_assigned_mode_summary(caa_lhr_mapped_v4_corrected)

,LASAM Mode,POP,prop
0,BR Coach (RailAir Coach),153139.76,0.25
1,Bus/Coach,4872065.06,7.95
2,Charter Coach,696959.33,1.14
3,Crossrail,7566175.10,12.35
4,Heathrow Express,2862047.20,4.67
5,Kiss-and-Fly,10254492.60,16.74
6,London Underground,8981204.10,14.66
7,Other,171123.60,0.28
8,Park-and-Fly,6864308.17,11.21
9,Taxi,18829465.96,30.74


In [49]:
get_assigned_mode_summary(caa_lhr_mapped_v5)

,LASAM_Mode,POP,prop
0,BR Coach (RailAir Coach),163551.41,0.27
1,Bus/Coach,4828224.03,7.88
2,Charter Coach,679686.48,1.11
3,Crossrail,7617816.15,12.44
4,Heathrow Express,2790438.80,4.56
5,Kiss-and-Fly,9907326.72,16.18
6,London Underground,8980198.18,14.66
7,Other,172341.54,0.28
8,Park-and-Fly,7244254.98,11.83
9,Taxi,18865125.65,30.80


In [50]:
get_assigned_mode_summary(caa_lhr_mapped_v6)

,LASAM Mode,POP,prop
0,BR Coach (RailAir Coach),163551.41,0.27
1,Bus/Coach,4918581.76,8.03
2,Charter Coach,698040.42,1.14
3,Crossrail,7715794.71,12.59
4,Heathrow Express,2777220.63,4.53
5,Kiss-and-Fly,9744325.71,15.90
6,London Underground,9048163.85,14.77
7,Other,145289.55,0.24
8,Park-and-Fly,7063794.07,11.53
9,Taxi,18995289.84,31.00


In [51]:
get_assigned_mode_summary(caa_lhr_mapped_v6_old_lasam_mode_lu)

,LASAM_Mode,POP,prop
0,BR Coach (RailAir Coach),163551.41,0.27
1,Bus/Coach,4918581.76,8.03
2,Charter Coach,698040.42,1.14
3,Crossrail,7702576.53,12.57
4,Heathrow Express,2790438.80,4.55
5,Kiss-and-Fly,9906694.62,16.17
6,London Underground,9048163.85,14.77
7,Other,153206.96,0.25
8,Park-and-Fly,7246986.13,11.83
9,Taxi,18641811.45,30.43
